In [1]:
import pyranges as pr
import pandas as pd
import pyranges.genomicfeatures as gf
from Bio import SeqIO
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
species = "dsim" 
strain = "w501"
# tile size
dist = 1000
genome_file = "../run_model_dsim/dsim-all-chromosome-r2.02.fasta"
suffix = "sim" 

#### Load peaks file ... output of macs2

In [9]:
peaks_testis = pd.read_table('../peaks_testis/' + species + '/' + strain + '/' + strain + '_peaks.narrowPeak', header=None)
peaks_head = pd.read_table('../peaks_head/' + species + '/' + strain + '_peaks.narrowPeak', header=None)

In [10]:
peaks_testis

,0,1,2,3,4,5,6,7,8,9
0,211000022278032,4,527,2057_peak_1,346117,.,30.35265,34616.90234,34611.74609,383
1,211000022278033,26,957,2057_peak_2,2192,.,7.69275,221.61166,219.22307,250
2,211000022278038,0,316,2057_peak_3,117772,.,32.28075,11781.66406,11777.24512,163
3,211000022278047,1400,2020,2057_peak_4,577139,.,45.10109,57719.66406,57713.91797,506
4,211000022278049,0,222,2057_peak_5,103527,.,48.83273,10357.05957,10352.70703,81
...,...,...,...,...,...,...,...,...,...,...
16052,rDNA,70978,71358,2057_peak_16053,1148,.,1.70783,116.90473,114.88138,286
16053,rDNA,71620,73131,2057_peak_16054,6955,.,2.98461,698.90240,695.58466,260
16054,rDNA,73504,74984,2057_peak_16055,1884,.,2.02668,190.70396,188.41321,94
16055,rDNA,75266,75840,2057_peak_16056,257,.,1.39328,27.28363,25.74117,168


In [11]:
peaks_head[peaks_head[0].isin(['X', '3R', '2L', '2R', '4', '3L'])]

,0,1,2,3,4,5,6,7,8,9
132,2L,222,1453,2057_peak_133,438,.,3.03676,45.30350,43.81658,602
133,2L,1626,1853,2057_peak_134,187,.,2.17714,20.12921,18.79195,149
134,2L,2051,2311,2057_peak_135,260,.,2.41573,27.39224,26.00238,190
135,2L,2512,2762,2057_peak_136,194,.,2.19419,20.78481,19.44180,171
136,2L,2996,3234,2057_peak_137,213,.,2.24713,22.73995,21.38298,163
...,...,...,...,...,...,...,...,...,...,...
20420,X,23527421,23527773,2057_peak_20421,71,.,1.86266,8.36594,7.15191,65
20421,X,23535299,23535821,2057_peak_20422,1233,.,4.58406,125.07506,123.31992,167
20422,X,23536100,23536487,2057_peak_20423,10313,.,16.15565,1034.52026,1031.39819,111
20423,X,23537208,23537509,2057_peak_20424,527,.,3.30062,54.25665,52.72941,259


In [12]:
def extr_startend(peaks, tis):    
    peaks = peaks[[0 ,1 ,2, 3]]
    peaks = peaks.rename(columns={0: "chr", 1: "start", 2: "end", 3:"peak_name"})
    peaks['peak_name'] = peaks['peak_name'] + tis
    peaks['chr'] = peaks['chr'].str.replace('Scf_','')
    return peaks
peaks_testis = extr_startend(peaks_testis, 'testis' )
peaks_head = extr_startend(peaks_head, 'head' )

In [13]:
peaks_testis

,chr,start,end,peak_name
0,211000022278032,4,527,2057_peak_1testis
1,211000022278033,26,957,2057_peak_2testis
2,211000022278038,0,316,2057_peak_3testis
3,211000022278047,1400,2020,2057_peak_4testis
4,211000022278049,0,222,2057_peak_5testis
...,...,...,...,...
16052,rDNA,70978,71358,2057_peak_16053testis
16053,rDNA,71620,73131,2057_peak_16054testis
16054,rDNA,73504,74984,2057_peak_16055testis
16055,rDNA,75266,75840,2057_peak_16056testis


In [14]:
peaks_head

,chr,start,end,peak_name
0,211000022278032,14,484,2057_peak_1head
1,211000022278038,15,267,2057_peak_2head
2,211000022278047,1411,2016,2057_peak_3head
3,211000022278089,59,341,2057_peak_4head
4,211000022278090,71,248,2057_peak_5head
...,...,...,...,...
20479,rDNA,67782,68240,2057_peak_20480head
20480,rDNA,68475,69559,2057_peak_20481head
20481,rDNA,71011,71305,2057_peak_20482head
20482,rDNA,71742,73206,2057_peak_20483head


Function to identify peak center

In [15]:
peaks_testis

,chr,start,end,peak_name
0,211000022278032,4,527,2057_peak_1testis
1,211000022278033,26,957,2057_peak_2testis
2,211000022278038,0,316,2057_peak_3testis
3,211000022278047,1400,2020,2057_peak_4testis
4,211000022278049,0,222,2057_peak_5testis
...,...,...,...,...
16052,rDNA,70978,71358,2057_peak_16053testis
16053,rDNA,71620,73131,2057_peak_16054testis
16054,rDNA,73504,74984,2057_peak_16055testis
16055,rDNA,75266,75840,2057_peak_16056testis


In [16]:
def pyr_peaks(peaks):
    gr_peaks = pr.from_dict({"Chromosome": list(peaks.chr),
                       "Start": list(peaks.start), 
                       "End": list(peaks.end),
                        "Strand": "+",
                            "peak_name": list(peaks.peak_name)})
    return gr_peaks
gr_peaks_testis = pyr_peaks(peaks_testis)
gr_peaks_head = pyr_peaks(peaks_head)
gr_peaks_tot = pyr_peaks(pd.concat([peaks_testis, peaks_head]))

In [17]:
good_chrs = ['X', '2L', '2R', '3L', '3R', '4']

In [18]:
gr_peaks_head = gr_peaks_head[gr_peaks_head.Chromosome.isin(good_chrs)]
gr_peaks_testis = gr_peaks_testis[gr_peaks_testis.Chromosome.isin(good_chrs)]
gr_peaks_tot = gr_peaks_tot[gr_peaks_tot.Chromosome.isin(good_chrs)]
# merge peaks that overlap in both tissues
gr_peaks_tot = gr_peaks_tot.merge()

In [19]:
mpeaks_testis = gr_peaks_tot.overlap(gr_peaks_testis)
mpeaks_head = gr_peaks_tot.overlap(gr_peaks_head)

In [20]:
mpeaks_head

,Chromosome,Start,End,Strand
0,2L,210,5038,+
1,2L,5529,5973,+
2,2L,17830,18993,+
3,2L,20660,21838,+
4,2L,34807,35119,+
...,...,...,...,...
19855,X,23527401,23527849,+
19856,X,23535299,23535821,+
19857,X,23536100,23536569,+
19858,X,23537191,23537632,+


In [27]:
mpeaks_head.df['Chromosome'].value_counts()

3R    4598
3L    4021
X     3834
2R    3648
2L    3583
4      176
Name: Chromosome, dtype: int64

In [25]:
mpeaks_head.df

,Chromosome,Start,End,Strand
0,2L,210,5038,+
1,2L,5529,5973,+
2,2L,17830,18993,+
3,2L,20660,21838,+
4,2L,34807,35119,+
...,...,...,...,...
19855,X,23527401,23527849,+
19856,X,23535299,23535821,+
19857,X,23536100,23536569,+
19858,X,23537191,23537632,+


DF containing positive peak examples -- coordinates

In [14]:
def get_peakctr(df, flankd):
    df['mp'] = (df.Start + df.End)/2
    df['rstart'] = np.ceil(df.mp - flankd)
    df['rend'] = np.ceil(df.mp + flankd)
    return df

def get_pospeaks(gr_peaks):
    peaks = gr_peaks.df
    peaks = get_peakctr(peaks, dist/2)
    pos_peaks = peaks[["Chromosome", "rstart", "rend"]]
    pos_peaks = pos_peaks.rename(columns={"chr": "Chromosome", "rstart": "Start", "rend": "End"})
    return pos_peaks
pos_peaks_head = get_pospeaks(mpeaks_head)
pos_peaks_testis = get_pospeaks(mpeaks_testis)

In [15]:
pos_peaks_head['label_head'] = 1
pos_peaks_testis['label_testis'] = 1
pos_peaks = pd.merge(pos_peaks_head, pos_peaks_testis, on=['Chromosome', 'Start', 'End'], how="outer")
pos_peaks['label_head'] = pos_peaks['label_head'].fillna(0)
pos_peaks['label_testis'] = pos_peaks['label_testis'].fillna(0)

In [16]:
pos_peaks[pos_peaks.Start == 15778.0]

,Chromosome,Start,End,label_head,label_testis


In [17]:
sum(pos_peaks.label_head == 1.0)

17557

In [18]:
print(mpeaks_head)
print(mpeaks_testis)

+--------------+-----------+-----------+--------------+
| Chromosome   | Start     | End       | Strand       |
| (category)   | (int32)   | (int32)   | (category)   |
|--------------+-----------+-----------+--------------|
| 2L           | 25        | 560       | +            |
| 2L           | 1808      | 3185      | +            |
| 2L           | 3384      | 3638      | +            |
| 2L           | 4101      | 4540      | +            |
| ...          | ...       | ...       | ...          |
| X            | 21736905  | 21737980  | +            |
| X            | 21742825  | 21743224  | +            |
| X            | 21744730  | 21745203  | +            |
| X            | 21746697  | 21746932  | +            |
+--------------+-----------+-----------+--------------+
Stranded PyRanges object has 17,557 rows and 4 columns from 6 chromosomes.
For printing, the PyRanges was sorted on Chromosome and Strand.
+--------------+-----------+-----------+--------------+
| Chromosome   | Star

In [19]:
pos_peaks_testis

,Chromosome,Start,End,label_testis
0,2L,-207.0,793.0,1
1,2L,1997.0,2997.0,1
2,2L,3821.0,4821.0,1
3,2L,4594.0,5594.0,1
4,2L,15439.0,16439.0,1
...,...,...,...,...
15671,X,21620632.0,21621632.0,1
15672,X,21638618.0,21639618.0,1
15673,X,21730815.0,21731815.0,1
15674,X,21731778.0,21732778.0,1


### Load reference genome

In [20]:
chrs = {}
with open(genome_file, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        key = record.id.replace('Scf_', '')
        chrs[key] = record.seq

In [21]:
len(chrs['3L'])

24197627

### Create intervals
need this for negative examples

In [22]:
#need this for negative examples
def create_intervals(chrom_list, tile_size, chrs):
    length_dict = {c:len(chrs[c]) for c in good_chrs}
    start = [0]*len(chrom_list)
    end = [len(chrs[c]) for c in chrom_list]
    gr = pr.from_dict({"Chromosome": chrom_list, "Start": start, "End": end})
    tiled = pr.gf.tile_genome(gr, tile_size)
    return tiled



In [23]:
# this generates labels with intervals
def generate_labels(chrom_list, 
                    tile_size, 
                    chrs, 
                    mpeaks_head,
                    mpeaks_testis,
                    pos_peaks):
    
    # select positive examples
    pos_peaks = pos_peaks.copy()
    pos_peaks = pos_peaks[pos_peaks.Chromosome.isin(chrom_list)]
    
    
    ranges = create_intervals(chrom_list, tile_size, chrs)
    ranges = ranges.as_df()
    #ranges['idx'] = ranges.index
    ranges['label_head'] = 0 
    ranges['label_testis'] = 0 
    ranges = pr.PyRanges(ranges)
    ranges = ranges.overlap(mpeaks_head, invert=True)
    ranges = ranges.overlap(mpeaks_testis, invert=True)
    ranges = ranges.df
    #neg_samples = ranges[ranges.label == 0]
    neg_samples = ranges[["Chromosome", "Start", "End", "label_head", "label_testis"]]
    
    out = pd.concat([neg_samples, pos_peaks])
    return out.sample(frac=1 , random_state=10) # to shuffle


In [24]:
def extract_sequence(chrs, chrom, start, end):
    try: 
        
        seq = chrs[chrom][start:end]
    except:
        
        seq = "bad"
    # return lowercase to keep consistent
    return str(seq).lower()

In [25]:
def one_hot(seq):
    encode = np.zeros([4, len(seq)], dtype=np.float32)
    for i, nuc in enumerate(seq):
        if nuc == 'a':
            encode[0, i] = 1
        elif nuc == 't':
            encode[1, i] = 1
        elif nuc == 'c':
            encode[2, i] = 1
        elif nuc == 'g':
            encode[3, i] = 1
    return encode

In [26]:
# make data sized (N, C, L) where L is the length of sequence, C is the 4 channels
# and N is the number of datapoints
#genel = list(tss.gene_id)
def get_seqdata(df, chrs):    
    seqdata = np.zeros([len(df.Start),4, dist], dtype=np.float32)
    y = np.zeros([len(df.Start), 2])
    #extdata = np.zeros([len(genel),2], dtype=np.float32)
    df_out_l = [0]*len(df.Start)
    i = 0
    gene_order = list()
    for index, row in df.iterrows():
        #print(index)
        chrom = row.Chromosome
        start = row.Start
        end = row.End

        seq = extract_sequence(chrs, chrom, int(start), int(end))
        if len(seq) == dist:
            onehot = one_hot(seq)
            seqdata[i, :, :] = onehot
            y[i, 0] = row.label_head
            y[i, 1] = row.label_testis
            df_out_l[i] = row
            i += 1
    # remove any genes with all zeros. Code is a bit convoluted
    # rows with all zeros didnt have the proper length 
    # input sequences
    idx = (np.sum(np.sum(seqdata,axis=2), axis=1) != 0)
    X = seqdata[idx]
    y = y[idx]
    df_out_l = [df_out_l[i] for i in range(len(df_out_l)) if idx[i]]
    df_out = pd.concat(df_out_l, axis=1).transpose()
    #df_out = df_out.iloc[idx]
    return (X, y, df_out)



In [27]:
data_train = generate_labels(['X', '3R', '2L', '2R', '4'], dist, chrs, mpeaks_head, mpeaks_testis, pos_peaks)
X_train, y_train, data_train = get_seqdata(data_train, chrs)

data_test = generate_labels(['3L'], dist, chrs, mpeaks_head, mpeaks_testis, pos_peaks)
X_test, y_test, data_test = get_seqdata(data_test, chrs)
#save test data coordinates (need for later)
data_test.to_csv('../run_model_dsim/2test_coors_'+ suffix)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=10)


In [28]:
X_t.shape

(22388, 4, 1000)

In [29]:


np.save('../run_model_dsim/2X_train2L', X_train, allow_pickle = False)
np.save('../run_model_dsim/2X_test2L', X_test, allow_pickle = False)
np.save('../run_model_dsim/2X_val2L', X_val, allow_pickle = False)

np.save('../run_model_dsim/2y_train2L', y_train, allow_pickle = False)
np.save('../run_model_dsim/2y_test2L', y_test, allow_pickle = False)
np.save('../run_model_dsim/2y_val2L', y_val, allow_pickle = False)
    